## 0.0 read data

In [261]:
import numpy as np
from numpy import linalg
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import time
from datetime import *
import copy

In [262]:
df_origin = pd.read_csv('./data/train.csv')
label = pd.read_csv('./data/train_label.csv')
df_test = pd.read_csv('./data/test.csv')

## concate

In [263]:
y_isCan = df_origin['is_canceled']
y_adr = df_origin['adr']
df_origin = df_origin.drop(columns=['reservation_status', 'reservation_status_date', 'is_canceled', 'adr'])

In [264]:
concate = pd.concat([df_origin,df_test],axis=0).reset_index(drop = True)


In [265]:
concate.isna().sum()

ID                                     0
hotel                                  0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

### dropna

In [266]:
concate = concate.drop(columns=['country', 'agent', 'company'])

### country填median

In [267]:
concate = concate.fillna(concate.children.median())

In [268]:
concate['arrival_date_month'] = concate['arrival_date_month'].map({'January':1, 'February': 2, 'March':3, 'April':4, 'May':5, 'June':6, 'July': 7, 'August':8, 'September':9, 'October': 10, 'November': 11, 'December':12})
concate.arrival_date_month = concate.arrival_date_month.astype(int)

### 加星期幾

In [269]:
def dateToWeekDay(df):
    year = df.arrival_date_year
    month = df.arrival_date_month
    day = df.arrival_date_day_of_month
    n = len(df)
    weekDays = [0 for i in range(n)]
    for i in range(n):
        weekDays[i] = date(year[i], month[i], day[i]).weekday()   
    # return 1~7
    df1 = copy.deepcopy(df)
#     df1 = df1.rename(columns = {'arrival_date_year': 'year', 'arrival_date_month': 'month'})
    df1.insert(6,'weekDay', weekDays)
    df1 = pd.get_dummies(df1, columns = ['weekDay'])
    print(df1.columns)
    df1 = df1.drop(columns = ['weekDay_6']) # remove last one
    return df1;

In [270]:
concate = dateToWeekDay(concate)

Index(['ID', 'hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces',
       'total_of_special_requests', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'weekDay_6'],
      dtype='object')


### getdummy

In [271]:
# Creating dummy variables:
concate = pd.get_dummies(concate, columns=['hotel', 'arrival_date_month', 'meal', 'market_segment', 'distribution_channel', \
                                 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'])
print(concate.columns)

Index(['ID', 'lead_time', 'arrival_date_year', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'hotel_City Hotel',
       'hotel_Resort Hotel', 'arrival_date_month_1', 'arrival_date_month_2',
       'arrival_date_month_3', 'arrival_date_month_4', 'arrival_date_month_5',
       'arrival_date_month_6', 'arrival_date_month_7', 'arrival_date_month_8',
       'arrival_date_month_9', 'arrival_date_month_10',
       'arrival_date_month_11', 'arrival_date_month_12', 'meal_BB', 'meal_FB',
       'meal_HB', 'meal_SC', 'meal_Undefined', 'market_segment_Aviation',
       'market_segment_Complementary',

In [272]:
concate = concate.drop(columns=['hotel_Resort Hotel', 'arrival_date_month_9', 'meal_Undefined', 'market_segment_Undefined', 'distribution_channel_Undefined', 'reserved_room_type_P', 'assigned_room_type_P',  'deposit_type_Refundable', 'customer_type_Transient-Party'])

In [273]:
concate.isna().sum()

ID                           0
lead_time                    0
arrival_date_year            0
arrival_date_week_number     0
arrival_date_day_of_month    0
                            ..
deposit_type_No Deposit      0
deposit_type_Non Refund      0
customer_type_Contract       0
customer_type_Group          0
customer_type_Transient      0
Length: 75, dtype: int64

### 切成train, validate, test

In [274]:
# y_isCan, y_adr 要切train, val
y_train_isCan = y_isCan.loc[:74842]
y_train_adr = y_adr.loc[:74842]
y_val_isCan = y_isCan.loc[74843:]
y_val_adr = y_isCan.loc[74843:]

#concate 要切train, val, test
X_train = concate.loc[:74842, :]
X_val = concate.loc[74843:91530, :]
X_test = concate.loc[91530:, :]

In [275]:
print(len(y_train_isCan))
print(len(y_train_adr))
print(len(y_val_isCan))
print(len(y_val_adr))
print(len(X_train))
print(len(X_val))
print(len(X_test))

74843
74843
16688
16688
74843
16688
27860


### 找index/tree

df_2016 = df_origin[df_origin['arrival_date_year'] == 2016]
df_201611 = df_2016[df_2016['arrival_date_month'] == 11]
df_20161130 = df_201611[df_201611['arrival_date_day_of_month'] == 30]

df_20161130

Split data at index 74843

:74842 -> train

74843: -> val

df_tree = df_origin.fillna(df.children.median())
df_tree = df_tree.drop(columns = ['country', 'agent', 'company', 'reservation_status_date'])

df_tree.dtypes 

# meal, market_segment, distribution_channel, reserved_room_type, assigned_room_type, deposit_type, customer_type
df_tree['meal'] = df_tree['meal'].map({'BB':1, 'FB': 2, 'HB':3, 'SC':4, 'Undefined':5})
df_tree['market_segment'] = df_tree['market_segment'].map({'Aviation':1, 'Complementary': 2, 'Corporate':3, 'Direct':4, 'Groups':5, 'Offline TA/TO':6, 'Online TA':7, 'Undefined': 8})
df_tree['distribution_channel'] = df_tree['distribution_channel'].map({'Corporate':1, 'Direct': 2, 'GDS':3, 'TA/TO':4, 'Undefined':5})
df_tree['reserved_room_type'] = df_tree['reserved_room_type'].map({'A':1, 'B': 2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'L':9, 'P':10})
df_tree['assigned_room_type'] = df_tree['assigned_room_type'].map({'A':1, 'B': 2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'L':9, 'P':10, 'I':11, 'K':12})
df_tree['deposit_type'] = df_tree['deposit_type'].map({'No Deposit':1, 'Non Refund': 2, 'Refundable':3})
df_tree['customer_type'] = df_tree['customer_type'].map({'Contract':1, 'Group': 2, 'Transient':3, 'Transient-Party':4})

df_tree['meal']

df_tree.dtypes 

df_tree.isna().sum()

train_tree = df_tree.loc[:74842, :]
val_tree = df_tree.loc[74843:, :]

X_train_tree = train_tree.drop(columns = ['is_canceled', 'adr'])
y_train_tree_isCan = train_tree['is_canceled']
y_train_tree_adr = train_tree['adr']
X_val_tree = val_tree.drop(columns = ['is_canceled', 'adr'])
y_val_tree_isCan = val_tree['is_canceled']
y_val_tree_adr = val_tree['adr']

## Basic analysis

#### Target: "is_canceled"

Steps:

1. Predict "is_canceled" by attributes in each tuple

2. Multiply predictions by "ADR"

3. Group by date

4. Predict classification of the daily revenue to the labels on $y$.


#### Building X and y

#### Splitting train/validation
(isCan/adr)

Split data at index 74843

:74842 -> train

74843: -> val

In [59]:
y_train_isCan.value_counts(normalize=True)

0    0.638163
1    0.361837
Name: is_canceled, dtype: float64

The cancel rate is around 35%.

In [60]:
y_train_adr.value_counts()

0.000000      75
47.000000      5
31.000000      5
42.000000      5
16.000000      4
              ..
139.827646     1
79.667069      1
115.188575     1
151.865524     1
50.842307      1
Name: adr, Length: 74732, dtype: int64

### Computing accuracy 

In [262]:
from sklearn.metrics import accuracy_score
def computeAccuracy(X, y, y_result): #x: test revenue, y: test label
    print("Accuracy =", accuracy_score(y, y_result))
    print("Number of mislabeled points out of a total %d points : %d"% (X.shape[0],(y != y_result).sum()))
    print("E_val = ", (y != y_result).sum() / X.shape[0])
    
from sklearn.metrics import r2_score
def calculateRsquare(X, y, y_result): #x: test revenue, y: test label
    print("R square =", r2_score(y, y_result))

## Predict is_can

### KNN

In [54]:
def trainKNN(X_train_df, y_train, k, X_val_df, y_val):
    print("Pre-processing...")
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', 'days_in_waiting_list', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_feat.to_numpy()
    X_train_feat = X_train_df.to_numpy()
    knn = KNeighborsClassifier(n_neighbors = k)
    print("Fitting...")
    knn_best = knn.fit(X_train_feat, y_train)
#     X_val_feat = X_val_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_val_feat = X_val_df[['distribution_channel_GDS', 'reserved_room_type_B', 'reserved_room_type_L', 'assigned_room_type_B', 'assigned_room_type_L']]
#     X_val_feat = X_val_feat.to_numpy()
    X_val_feat = X_val_df.to_numpy()
    print("Predicting...")
    y_pred = knn_best.predict(X_val_feat)
    print("Calculating accuracy...")
    sc = knn_best.score(X_val_feat, y_val, sample_weight=None)
    return y_pred, sc;

In [55]:
y_pred, sc = train_KNN_isCan(X_train, y_train_isCan, 5, X_val, y_val_isCan)

NameError: name 'train_KNN_isCan' is not defined

In [185]:
sc

0.6679050814956855

In [187]:
computeAccuracy(X_val, y_val_isCan, y_pred)

Accuracy = 0.6679050814956855
Number of mislabeled points out of a total 16688 points : 5542
E_val =  0.3320949185043145


### Desicion Stump

In [315]:
from sklearn import tree

def trainDecisionStump(X_train, y_train, X_val):
    clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

def trainDecisionStumpRegressor(X_train, y_train, X_val):
    clf = tree.DecisionTreeRegressor(max_depth=10).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [81]:
y_result = trainDecisionStump(X_train, y_train_isCan, X_val)

In [82]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.6718600191754555
Number of mislabeled points out of a total 16688 points : 5476
E_val =  0.3281399808245446


#### tree isCan

In [260]:
y_result = trainDecisionStump(X_train_tree, y_train_tree_isCan, X_val_tree)

In [263]:
computeAccuracy(X_val_tree, y_val_tree_isCan, y_result)

Accuracy = 0.652984180249281
Number of mislabeled points out of a total 16688 points : 5791
E_val =  0.3470158197507191


#### tree adr

In [316]:
y_result = trainDecisionStumpRegressor(X_train_tree, y_train_tree_adr, X_val_tree)

In [317]:
calculateRsquare(X_val_tree, y_val_tree_adr, y_result)

R square = 0.4756839274931203


### Logistic Regression

使用 L2 regularization, 搭配上'sag'solver

In [117]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


### solver: lbfgs, newton-cg, sag
def trainLogisticRegression(X_train, y_train, X_val):
    clf = LogisticRegression(random_state=0, penalty='l2', solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [118]:
y_result = trainLogisticRegression(X_train, y_train_isCan, X_val)

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [119]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7870925215723873
Number of mislabeled points out of a total 16688 points : 3553
E_val =  0.21290747842761265


### SVM

In [ ]:
import numpy as np
import sys
sys.path.insert(1, '.')
from svmutil import *

In [ ]:
def toSVMdata(X, y): 
    N, d = X_train.shape
    X_svm = []
    for n in range(N):
        X_svm.append({})
        for i in range(d):
            X_svm[n].update({i: X_train[n,i]}) 
    y_svm = list(y)
    return X_svm, y_svm

In [ ]:
def SVM (Xtrain, ytrain, Xval, yval):
    bestaccu = 0
    for i in range(-2, 3):
        c = pow(10, i)
        # turn into SVM data form
        Xval, yval = toSVMdata(Xval, yval)
        Xtrain, ytrain = toSVMdata(Xtrain, ytrain)
        # train and prediction
        model = svm_train(ytrain, Xtrain, '-s 0 -t 0 -c {C}'.format(C = c))
        print("with C =", c, ",", end = "")
        p_label, p_acc, p_val = svm_predict(yval, Xval, model)
        if (p_acc[0] > bestaccu):
            bestaccu = p_acc[0]
    bestaccu /= 100
    return bestaccu

In [ ]:
a = X_train.to_numpy()
b = y_train_isCan.to_numpy()
print(type(a))
print(type(b))
X_train_svm, y_train_svm = toSVMdata(X_train.to_numpy(), y_train_isCan.to_numpy())
X_val_svm, y_val_svm = toSVMdata(X_val, y_val_isCan)
accuracy = SVM(X_train_svm, y_train_svm, X_val_svm, y_val_svm)
print("E_val =", 1 - accuracy)

### GussianNB

In [83]:
from sklearn.naive_bayes import GaussianNB

def trainGussianNB(X_train, y_train, X_val):
    clf = GaussianNB().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [84]:
y_result = trainGussianNB(X_train, y_train_isCan, X_val)

In [85]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7596476510067114
Number of mislabeled points out of a total 16688 points : 4011
E_val =  0.24035234899328858


## Predict adr

### Neural Network

In [117]:
from sklearn.neural_network import MLPClassifier

def trainNeural(X_train, y_train, X_val):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [131]:
# y_result = trainNeural(X_train, y_train_adr, X_val)

In [ ]:
# calculateRsquare(X_val, y_val_adr, y_result)

### Linear Regression

In [70]:
from sklearn.linear_model import LinearRegression

def trainLinearRegression(X_train_adr, y_train_adr, X_val_adr):
    clf = LinearRegression().fit(X_train_adr, y_train_adr)
    y_result = clf.predict(X_val_adr)
    return(y_result)

In [71]:
y_result = trainLinearRegression(X_train, y_train_adr, X_val)

In [72]:
calculateRsquare(X_val, y_val_adr, y_result)

R square = 0.44109509968084737


### AdaboostClassifier (for is_canceled)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification


def AdaboostClassifier(x, y):
    clf = AdaBoostClassifier(n_estimators=100, random_state=0, algorithm='SAMME')
    clf.fit(x, y) 
    y_result = clf.predict(x)
    return(y_result)
    

### Calculate predicted daily revenue

In [106]:
def calculateDateRevenue(df):
    df1 = df[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "adr", "is_canceled", "stays_in_weekend_nights", "stays_in_week_nights"]]
    df1["revenue"] = (1 - df1["is_canceled"]) * df1["adr"] * (df1["stays_in_week_nights"] + df1["stays_in_weekend_nights"])
    df2 = df1[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "revenue"]]
    df3 = df2.groupby(by=["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]).sum()
    x = df3.to_numpy()
    return(x) # one column: revenue

## Revenue to label 

In [107]:
df_train.columns

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date'],
      dtype='object')

In [108]:
X_train_revenue = calculateDateRevenue(df_train)
# print(len(X_train_revenue))
X_val_revenue = calculateDateRevenue(df_val)
# print(len(X_val_revenue))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
label["ID"] = label.index
label_train = label.loc[:518, :]
label_val = label.loc[519:, :]

In [110]:
y_train_label = label_train["label"].to_numpy()
# print(len(y_train_label))
y_val_label = label_val["label"].to_numpy()
# print(len(y_val_label))

for i in range(len(X_train_revenue)):
    print(X_train_revenue[i], y_train_label[i])

### KNN

In [204]:
y_result = trainKNN(X_train_revenue, y_train_label, 5, X_val_revenue, y_val_label)

Pre-processing...


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [116]:
computeAccuracy(X_val_revenue, y_val_revenue, y_result)

### Desicion Stump 

In [111]:
y_result = trainDecisionStump(X_train_revenue, y_train_label, X_val_revenue)

In [112]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9917355371900827
Number of mislabeled points out of a total 121 points : 1
E_val =  0.008264462809917356


### Logistic Regression

In [113]:
y_result = trainLogisticRegression(X_train_revenue, y_train_label, X_val_revenue)

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [114]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9256198347107438
Number of mislabeled points out of a total 121 points : 9
E_val =  0.0743801652892562


### SVM

In [105]:
y_result = trainSVM(X_train_revenue, y_train_label, X_val_revenue)

In [78]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.38016528925619836
Number of mislabeled points out of a total 121 points : 75
E_val =  0.6198347107438017


### GuassianNB

In [115]:
y_result = trainGussianNB(X_train_revenue, y_train_label, X_val_revenue)

In [116]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9504132231404959
Number of mislabeled points out of a total 121 points : 6
E_val =  0.049586776859504134


## Test

In [ ]:
isCan = trainLogisticRegression(X_train, y_train_isCan, X_test)
